# Interview Pattern Example

This notebook demonstrates the **Interview Pattern**, a powerful prompt engineering technique where the AI controls the flow of information.

**The Concept:**
Instead of the user dumping all information at once, the AI acts as an interviewer. It asks specific questions one by one to gather the necessary context before providing a final recommendation.

**Key Mechanics:**
1.  **Persona**: The AI adopts a specific role (e.g., Career Coach).
2.  **Iterative Loop**: It asks one question, waits for an answer, and repeats.
3.  **State Management**: Since the OpenAI API is "stateless" (it doesn't remember past messages), we manually manage the conversation history.
4.  **Stop Sequence**: We define a specific tag (`<FINAL_ANSWER>`) that the AI uses to signal it has enough information to finish the interview.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

First, we import the necessary libraries and load your OpenAI API key. This setup ensures we can communicate with the model.

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import time

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

# Model configuration - can be overridden via environment variable
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

client = OpenAI(api_key=api_key)
print(f"OpenAI client ready! Using model: {MODEL}")

### 2. The System Prompt (The "Brain")

The **System Prompt** is the most critical part. It defines the rules of the "game."

We explicitly instruct the model to:
* Ask **only one** question at a time.
* Wait for the user's response.
* **Never** give advice prematurely.
* Use a specific **tag** (`<FINAL_ANSWER>`) when it's ready to conclude.

This prevents the model from rambling or trying to solve the problem with insufficient data.

In [ ]:
SYSTEM_PROMPT = """
You are an expert career coach helping a candidate improve their resume and prepare for interviews.

You must use the Interview Pattern technique strictly:
1. Ask ONLY ONE clear question at a time.
2. Wait for the user's answer before asking the next.
3. Remember everything said so far.
4. When you have enough information to give excellent personalized advice, respond with exactly:
   <FINAL_ANSWER>
   followed by your full recommendation.
5. Never give advice early.

Start by greeting the user and asking your first question.
"""

### 3. Initializing the Conversation

Because the OpenAI API is **stateless**, we must keep a running list of all messages (the `messages` list).

**The "Hidden Trigger" Trick:**
The model usually waits for the user to speak first. To make the AI take the lead (greet us and ask the first question), we insert a hidden "trigger" message from the user side. We add this to the history but don't show it in the output, creating the illusion that the AI started the conversation on its own.

In [ ]:
# Initialize history with the System Prompt
messages = [{"role": "system", "content": SYSTEM_PROMPT}]

# Maximum number of messages to keep (prevents token limit issues)
MAX_HISTORY = 20

def trim_message_history(messages, max_messages=MAX_HISTORY):
    """Keep system prompt and last N-1 messages to prevent token overflow."""
    if len(messages) > max_messages:
        return [messages[0]] + messages[-(max_messages - 1):]
    return messages

print("Career Coach Interview Started! Type 'quit' to stop.\n")

# Inject a hidden user message to kickstart the AI's persona
initial_trigger = "Hello, please begin the career coaching interview."
messages.append({"role": "user", "content": initial_trigger})

try:
    # Get the Coach's first response (Greeting + Question 1)
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.7
    )
    reply = response.choices[0].message.content

    # Save the AI's reply to history
    messages.append({"role": "assistant", "content": reply})

    print(f"Coach: {reply}\n")
    
except Exception as e:
    print(f"Error starting interview: {e}")

### 4. The Interview Loop

This is the main interactive engine.

**What happens here:**
1.  **Input:** We accept your answer to the coach's question.
2.  **History Update:** We append your answer to the `messages` list.
3.  **Generation:** We send the *entire* history back to OpenAI to get the next question.
4.  **Stop Check:** We check the AI's response for the `<FINAL_ANSWER>` tag.
    * If found: We strip the tag, print the final advice, and break the loop.
    * If not found: The interview continues.

In [ ]:
while True:
    # 1. Get User Input
    user_input = input("You: ")
    
    if user_input.strip().lower() in ["quit", "exit", "bye", "stop"]:
        print("\nGood luck with your job hunt!")
        break
    
    if not user_input.strip():
        print("Please enter a response.")
        continue
    
    # 2. Append User Input to History
    messages.append({"role": "user", "content": user_input})
    
    try:
        # 3. Generate Next Response
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.7
        )
        reply = response.choices[0].message.content
        
        # 4. Append AI Response to History
        messages.append({"role": "assistant", "content": reply})
        
        # 5. Trim history to prevent token overflow
        messages = trim_message_history(messages)
        
        # 6. Check for Conclusion
        if "<FINAL_ANSWER>" in reply:
            print("\n" + "="*70)
            print("FINAL RECOMMENDATION")
            print("="*70)
            # Clean up the output by removing the control tag
            print(reply.replace("<FINAL_ANSWER>", "").strip())
            print("="*70)
            break
        else:
            # Continue the interview
            print(f"\nCoach: {reply}\n")
        
        time.sleep(0.3)
        
    except Exception as e:
        print(f"\nError: {e}\n")
        messages.pop()  # Remove the failed user message